In [1]:
import os
import typing
import cv2
import numpy as np
import tensorflow as tf

from local_paths import *
"""
local_paths defines local paths to the required folders,
the variables used for the paths are listed below:

IMAGE_DATASET_128_DIR
IMAGE_DATASET_256_DIR
OUTPUT_DATASET_DIR_128
OUTPUT_DATASET_DIR_256
"""

2025-04-17 23:28:28.813727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744921708.843247  140239 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744921708.850689  140239 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744921708.870465  140239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744921708.870493  140239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744921708.870496  140239 computation_placer.cc:177] computation placer alr

'\nlocal_paths defines local paths to the required folders,\nthe variables used for the paths are listed below:\n\nIMAGE_DATASET_128_DIR\nIMAGE_DATASET_256_DIR\nOUTPUT_DATASET_DIR_128\nOUTPUT_DATASET_DIR_256\n'

In [ ]:
def dataset_generator(
        image_size: typing.Tuple[int, int], images_dir: bytes):
    
    images_dir = str(images_dir).split('\'')[1]
    # it is not specified what encoding is used, or I did not find it

    images_paths = [os.path.join(images_dir, name)
                    for name in os.listdir(images_dir)]
    for path in images_paths:

        image = cv2.imread(path)
        height, width, *_ = image.shape

        if (np.any((height, width) != image_size)):
            image = cv2.resize(image, image_size)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB).astype(np.float32)

        # normalization of L-channel (0, 255) -> (0, 1)
        l_channel = image[:, :, 0] / 255.0
        # add dimension (h, w) -> (h, w, 1)
        l_channel = l_channel[..., np.newaxis]

        # normalization (0, 255) -> (-1, 1)
        ab_channel = (image[:, :, 1:] - 128.0) / 128.0

        yield tf.convert_to_tensor(l_channel), tf.convert_to_tensor(ab_channel)


In [ ]:
IMAGE_SIZE = (128, 128)

dataset = tf.data.Dataset.from_generator(
    dataset_generator,
    args=(IMAGE_SIZE, IMAGE_DATASET_128_DIR), # converts strings to bytes
    output_signature=(
        tf.TensorSpec(shape=IMAGE_SIZE + (1,), dtype=tf.float32, name=None),
        tf.TensorSpec(shape=IMAGE_SIZE + (2,), dtype=tf.float32, name=None)))


W0000 00:00:1744922061.576963  140239 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [15]:
dataset.save(OUTPUT_DATASET_DIR_128)

In [ ]:
IMAGE_SIZE = (256, 256)

dataset = tf.data.Dataset.from_generator(
    dataset_generator,
    args=(IMAGE_SIZE, IMAGE_DATASET_256_DIR),
    output_signature=(
        tf.TensorSpec(shape=IMAGE_SIZE + (1,), dtype=tf.float32, name=None),
        tf.TensorSpec(shape=IMAGE_SIZE + (2,), dtype=tf.float32, name=None)))


In [ ]:
# dataset.save(OUTPUT_DATASET_DIR_256)